In [1]:
import approval_que_app as aqa

In [2]:
working_session = aqa.OracleConnect()
working_session_backup = aqa.TableBackUp(working_session)

# Create session backups
_result = working_session_backup.perform_session_backup()
print(_result)
# Display created file names
print(working_session_backup.session_bk_FORAQUS_file_name)
print(working_session_backup.session_bk_FORAQRC_file_name)
print(working_session_backup.session_bk_FTVAPPQ_file_name)


logs file exists in project_root
back_ups file exists in project_root
session_back_ups file exists in back_ups
user_back_ups file exists in back_ups
back_ups/session_back_ups/FORAQUS_2022-10-25
select  FORAQUS_QUEUE_ID, FORAQUS_USER_ID_APPR, FORAQUS_QUEUE_LEVEL, FORAQUS_QUEUE_LIMIT, FORAQUS_EFF_DATE, FORAQUS_NCHG_DATE, FORAQUS_TERM_DATE, FORAQUS_ACTIVITY_DATE, FORAQUS_USER_ID, FORAQUS_SURROGATE_ID, FORAQUS_VERSION, FORAQUS_DATA_ORIGIN, FORAQUS_VPDI_CODE from FORAQUS    
execute_sql_query  exec_time: 1.8296663761138916
select  FORAQRC_QUEUE_ID, FORAQRC_DOC_TYPE, FORAQRC_RULE_GROUP, FORAQRC_COAS_CODE, FORAQRC_FTYP_CODE, FORAQRC_FUND_CODE, FORAQRC_ORGN_CODE, FORAQRC_ATYP_CODE, FORAQRC_ACCT_CODE, FORAQRC_ACTIVITY_DATE, FORAQRC_USER_ID, FORAQRC_PROG_CODE, FORAQRC_SURROGATE_ID, FORAQRC_VERSION, FORAQRC_DATA_ORIGIN, FORAQRC_VPDI_CODE from FORAQRC    
execute_sql_query  exec_time: 0.2698194980621338
select  FTVAPPQ_QUEUE_ID, FTVAPPQ_DESCRIPTION, FTVAPPQ_QUEUE_LIMIT, FTVAPPQ_NEXT_QUEUE_ID, FTVA

In [ ]:
#returns who in the queues which they are in

_select = f"""select distinct FORAQUS_QUEUE_ID, FORAQUS_USER_ID_APPR,
listagg(distinct FORAQRC_ORGN_CODE,', '), listagg(distinct FTVORGN_TITLE,', ')"""
_from = f"""from FORAQUS, FORAQRC, FTVORGN"""
_where = f"""where FORAQUS_QUEUE_ID like '%B%'
and FORAQUS_NCHG_DATE is null
and FORAQUS_TERM_DATE is null
and replace(FORAQUS_QUEUE_ID, 'B', '') = replace(FORAQRC_QUEUE_ID, 'A', '')
and FTVORGN_ORGN_CODE = FORAQRC_ORGN_CODE"""
_by = f"""group by FORAQUS_QUEUE_ID, FORAQUS_USER_ID_APPR
order by FORAQUS_USER_ID_APPR, FORAQUS_QUEUE_ID """

x = working_session.db_query(cur,_select,_from,_where,_by)
for y in x:
    print(y)

In [ ]:
approver_un = 'SMILLER66'
approver_un = 'SMILLER66'
approver_un = 'SMILLER66'
approver_un = 'BJESTER1'
cur = working_session
_select = f"""foraqus.FORAQUS_QUEUE_ID,foraqus.FORAQUS_USER_ID_APPR,
(select count(FORAQUS_USER_ID_APPR)
from foraqus fa
where fa.FORAQUS_USER_ID_APPR != '{approver_un}'
      and fa.FORAQUS_QUEUE_ID = foraqus.FORAQUS_QUEUE_ID
                                and fa.FORAQUS_TERM_DATE is null
                                and fa.FORAQUS_NCHG_DATE is null) as OtherApprovers"""
_from = f"""foraqus"""
_where = f"""foraqus.FORAQUS_USER_ID_APPR = '{approver_un}'
and foraqus.FORAQUS_TERM_DATE is null
and foraqus.FORAQUS_NCHG_DATE is null"""
_order_by = """"""
_group_by = """"""
_max_records_returned = """"""

x = working_session.db_query(_select,_from,_where,_order_by,_group_by,_max_records_returned)
for y in x:
    print(y)

In [12]:
# search for empty queues
def search_for_mt_queues():
    mt_list = []
    _select = f"""foraqus.FORAQUS_QUEUE_ID,
    (select count(FORAQUS_USER_ID_APPR)
    from foraqus fa
    where fa.FORAQUS_QUEUE_ID = foraqus.FORAQUS_QUEUE_ID
                                and fa.FORAQUS_TERM_DATE is null
                                and fa.FORAQUS_NCHG_DATE is null) as OtherApprovers"""
    _from = f"""foraqus"""
    _where = f"""foraqus.FORAQUS_TERM_DATE is null
    and foraqus.FORAQUS_NCHG_DATE is null"""
    _order_by = """2"""
    _group_by = """"""
    _max_records_returned = """"""
    x = working_session.db_query(_select,_from,_where,_order_by,_group_by,_max_records_returned)
    for y in x:
        if y[1] == 0:
            print('alert', y)
            mt_list.append(y)
        print(y)
    return mt_list
search_for_mt_queues()

select  foraqus.FORAQUS_QUEUE_ID,
    (select count(FORAQUS_USER_ID_APPR)
    from foraqus fa
    where fa.FORAQUS_QUEUE_ID = foraqus.FORAQUS_QUEUE_ID
                                and fa.FORAQUS_TERM_DATE is null
                                and fa.FORAQUS_NCHG_DATE is null) as OtherApprovers from foraqus where foraqus.FORAQUS_TERM_DATE is null
    and foraqus.FORAQUS_NCHG_DATE is null order by 2  
execute_sql_query  exec_time: 0.49460577964782715
('099G', 1)
('101G', 1)
('102G', 1)
('105G', 1)
('103G', 1)
('107G', 1)
('108G', 1)
('110G', 1)
('113G', 1)
('114G', 1)
('117G', 1)
('118G', 1)
('043C', 1)
('046C', 1)
('094D', 1)
('121F', 1)
('047F', 1)
('014F', 1)
('141G', 1)
('073G', 1)
('075G', 1)
('119G', 1)
('121E', 1)
('124G', 1)
('125G', 1)
('130G', 1)
('132G', 1)
('133G', 1)
('135G', 1)
('136G', 1)
('089C', 1)
('134G', 1)
('040G', 1)
('051G', 1)
('052G', 1)
('053G', 1)
('054G', 1)
('055G', 1)
('058G', 1)
('063G', 1)
('064G', 1)
('065G', 1)
('090G', 1)
('093G', 1)
('098G', 1)
('

[]